In [348]:
import imagehash
from PIL import Image
import os
import os
import sys
sys.path.append('../../sharechat-scraper') # to access env variables
import pymongo
from pymongo import MongoClient
from s3_mongo_helper import initialize_mongo
import pandas as pd
import numpy as np
from datetime import date, datetime, timedelta
import wget
import logging
from time import perf_counter
import numpy as np
import sklearn
from sklearn.cluster import KMeans
import random
import scipy.spatial.distance

In [169]:
coll = initialize_mongo()
start = datetime(2020, 10, 5)
end = datetime(2020, 10, 12)
cursor = coll.find({"scraped_date": {'$gte':start,'$lt':end},
                    "media_type": "image"})

In [170]:
coll.count_documents({"scraped_date": {'$gte':start,'$lt':end},
                    "media_type": "image"})

18418

In [ ]:
start = perf_counter()
c=0
for i in cursor:
    try:
        wget.download(i["s3_url"], out="images/")
        c+=1
    except Exception as e: 
        print(e)
        continue
delta = perf_counter() - start
print(delta)

In [172]:
print(c)

10800


In [439]:
def get_hashes(path):
    hashfunc = imagehash.phash
    failed = []
    total = 0
    success = 0
    image_filenames = []
    image_filenames += [file for file in os.listdir(path)]
#     image_filenames += [os.path.join(userpath, path) for path in os.listdir(userpath)]
    images = {}
    for img in image_filenames:
        if img.split(".")[-1] in ["jpg", "jpeg", "png"]:
            try:
                phash = hashfunc(Image.open(os.path.join(path, img)))
                images[img] = str(phash)
                success +=1
            except Exception:
                failed.append(img)
                continue
            total +=1
    print("phash generated for {} of {} images".format(success, total))
    return images

In [440]:
start = perf_counter()
hashes = get_hashes(os.getcwd()+"/images")
delta = perf_counter() - start
print(delta)

phash generated for 10788 of 10788 images
127.05611668900383


In [464]:
df = pd.DataFrame.from_dict(hashes, orient="index", columns = ["phash"])
df.reset_index(inplace=True)
df.head(3)

,index,phash
0,1ee8da5f-326d-427d-85db-d54bea136ab0.jpg,989d1cded6d69610
1,75d01f9b-31c9-4c06-9ab0-c7968f3617da.jpg,f0a59e5e254b42bc
2,89c5d57c-c3c5-4359-bf68-ae830fceedcf.jpg,94971ef8694161b7


In [511]:
def add_url(filename):
    url = "https://s3.ap-south-1.amazonaws.com/sharechat-scraper.tattle.co.in/" + filename
    return url

In [525]:
df["img_urls"] = df["index"].map(add_url)

In [526]:
grouped = df.groupby(by="phash").agg({"index":"size", "img_urls":list})
grouped.rename(columns={'index':'count'}, inplace=True)
sorted = grouped.sort_values("count", ascending=False)
sorted.reset_index(inplace=True)

In [527]:
sorted.head(10)

,phash,count,img_urls
0,81d07fffe0611a85,34,[https://s3.ap-south-1.amazonaws.com/sharechat...
1,e3b24096fc0d3cad,20,[https://s3.ap-south-1.amazonaws.com/sharechat...
2,f8076bbc53817cc1,13,[https://s3.ap-south-1.amazonaws.com/sharechat...
3,85d3ba7ca5359a12,13,[https://s3.ap-south-1.amazonaws.com/sharechat...
4,aad2d00ed3392cb7,12,[https://s3.ap-south-1.amazonaws.com/sharechat...
5,ccbbe064401f7d59,11,[https://s3.ap-south-1.amazonaws.com/sharechat...
6,d225baeb205aeb25,9,[https://s3.ap-south-1.amazonaws.com/sharechat...
7,d7e7c00219199fd5,9,[https://s3.ap-south-1.amazonaws.com/sharechat...
8,f43f4f99609060de,9,[https://s3.ap-south-1.amazonaws.com/sharechat...
9,f788d82790fa2574,9,[https://s3.ap-south-1.amazonaws.com/sharechat...


In [543]:
# Narrow down to images that occur at least twice
len(sorted[sorted["count"] >= 2])

655

In [585]:
identical_clusters = sorted[sorted["count"] >= 2]

In [586]:
identical_clusters.reset_index(inplace=True,drop = True)

In [588]:
identical_clusters

,phash,count,img_urls
0,81d07fffe0611a85,34,[https://s3.ap-south-1.amazonaws.com/sharechat...
1,e3b24096fc0d3cad,20,[https://s3.ap-south-1.amazonaws.com/sharechat...
2,f8076bbc53817cc1,13,[https://s3.ap-south-1.amazonaws.com/sharechat...
3,85d3ba7ca5359a12,13,[https://s3.ap-south-1.amazonaws.com/sharechat...
4,aad2d00ed3392cb7,12,[https://s3.ap-south-1.amazonaws.com/sharechat...
...,...,...,...
650,a7b3586db72243b0,2,[https://s3.ap-south-1.amazonaws.com/sharechat...
651,cccc43ae34371e3c,2,[https://s3.ap-south-1.amazonaws.com/sharechat...
652,b038cec7e1c33738,2,[https://s3.ap-south-1.amazonaws.com/sharechat...
653,ea94944e5b998c3e,2,[https://s3.ap-south-1.amazonaws.com/sharechat...


In [555]:
# Find similar images
query = "81d07fffe0611a85"
for i, row in sorted.iterrows():
    # Convert hash hex string to original hash (binary array) 
    diff = imagehash.hex_to_hash(query) - imagehash.hex_to_hash(row["phash"])
    # phashes with Hamming distance less than 10 are likely to be similar images
    if diff <= 10:
        print(row["phash"])
        print(diff)
        print(row["img_urls"])

81d07fffe0611a85
0
['https://s3.ap-south-1.amazonaws.com/sharechat-scraper.tattle.co.in/fc0ae0b0-5f1e-4e02-b438-2e2e58603eb8.jpg', 'https://s3.ap-south-1.amazonaws.com/sharechat-scraper.tattle.co.in/b70f8fdd-3b47-4cc5-9194-e63a1836bb3e.jpg', 'https://s3.ap-south-1.amazonaws.com/sharechat-scraper.tattle.co.in/03163a95-2392-4088-ba7d-81cf9908edae.jpg', 'https://s3.ap-south-1.amazonaws.com/sharechat-scraper.tattle.co.in/b31c3138-c7b9-472a-899c-2f32c829eb5a.jpg', 'https://s3.ap-south-1.amazonaws.com/sharechat-scraper.tattle.co.in/ff25d508-30e9-49ac-8a40-eee2f7298b54.jpg', 'https://s3.ap-south-1.amazonaws.com/sharechat-scraper.tattle.co.in/2a29c11c-d4aa-49c5-953f-02510d548146.jpg', 'https://s3.ap-south-1.amazonaws.com/sharechat-scraper.tattle.co.in/a511dfd3-0035-4361-b250-39296ac54173.jpg', 'https://s3.ap-south-1.amazonaws.com/sharechat-scraper.tattle.co.in/b5b3c7b5-cdd6-43a4-b620-657f9b9bf925.jpg', 'https://s3.ap-south-1.amazonaws.com/sharechat-scraper.tattle.co.in/de07c84e-25e2-43fa-bc55-

In [589]:
identical_clusters.to_csv("phash_clusters.csv", index=False)

### Experiments

In [556]:
df["original_hash"] = df["phash"].map(imagehash.hex_to_hash)

In [333]:
arrays = np.array(df["original_hash"])

In [337]:
# Create pairwise (Hamming) difference matrix using numpy broadcasting
x = arrays[:, np.newaxis] - arrays[np.newaxis, :]
print(x)

[[0 28 32 ... 30 34 26]
 [28 0 22 ... 34 32 28]
 [32 22 0 ... 30 30 30]
 ...
 [30 34 30 ... 0 34 26]
 [34 32 30 ... 34 0 30]
 [26 28 30 ... 26 30 0]]


In [394]:
len(x)

10788

In [349]:
# Modify Imagehash library func to return array instead of ImageHash obj

def hex_to_hash(hexstr):
	"""
	Convert a stored hash (hex, as retrieved from str(Imagehash))
	back to a Imagehash object.
	Notes:
	1. This algorithm assumes all hashes are either
	   bidimensional arrays with dimensions hash_size * hash_size,
	   or onedimensional arrays with dimensions binbits * 14.
	2. This algorithm does not work for hash_size < 2.
	"""
	hash_size = int(np.sqrt(len(hexstr)*4))
	#assert hash_size == numpy.sqrt(len(hexstr)*4)
	binary_array = '{:0>{width}b}'.format(int(hexstr, 16), width = hash_size * hash_size)
	bit_rows = [binary_array[i:i+hash_size] for i in range(0, len(binary_array), hash_size)]
	hash_array = np.array([[bool(int(d)) for d in row] for row in bit_rows])
	return hash_array

In [575]:
# Calculate Hamming distance 
xor = hex_to_hash("87d03e7ee0259e85") ^ hex_to_hash("81d07fffe0611a85")
xor 

array([[False, False, False, False, False,  True,  True, False],
       [False, False, False, False, False, False, False, False],
       [False,  True, False, False, False, False, False,  True],
       [ True, False, False, False, False, False, False,  True],
       [False, False, False, False, False, False, False, False],
       [False,  True, False, False, False,  True, False, False],
       [ True, False, False, False, False,  True, False, False],
       [False, False, False, False, False, False, False, False]])

In [576]:
# Subract matching bits from total number of bits to get difference
np.count_nonzero(xor != 0)

10

In [577]:
imagehash.hex_to_hash("87d03e7ee0259e85") - imagehash.hex_to_hash("81d07fffe0611a85")

10

In [579]:
# Prepare hashes for kmeans clustering
arr = []
for i in df["phash"][:50]:
    arr.append(hex_to_hash(i).flatten())

In [580]:
arr[0].shape

(64,)

In [581]:
# How do you decide the number of clusters?
kmeans = KMeans(n_clusters=8)

In [582]:
kmeans.fit(arr)

KMeans()

In [583]:
# Are the clusters meaningful? 
kmeans.labels_ 

array([4, 4, 4, 2, 7, 6, 4, 1, 0, 1, 5, 5, 3, 0, 3, 0, 5, 7, 1, 3, 3, 0,
       3, 2, 2, 2, 2, 2, 2, 1, 6, 1, 5, 6, 1, 6, 7, 7, 4, 2, 3, 0, 3, 2,
       0, 5, 7, 4, 1, 3], dtype=int32)